# Configurações Iniciais

    - Importando bibliotecas necessárias

In [ ]:
import random
import os
import numpy as np
import torch
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
from tqdm import tqdm
import seaborn as sns

    - Definindo a seed
    - Setando o device

In [ ]:
def set_seed(seed=1234):
    random.seed(seed)
    os.environ['PYHTONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

set_seed()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

---

# Definindo o Dataset
    - Definindo as imagens
    - Realizando a normalização
    - Dividindo entre treino e teste

In [ ]:
img_size = (256,256)

transformations = transforms.Compose([
    transforms.Resize(img_size), 
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
dataset_dir = '.'
print(os.listdir(dataset_dir))

dataset = datasets.ImageFolder(root=dataset_dir, transform=transformations)

In [ ]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [ ]:
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

---

# Definindo Funções

    - Validação
    - Treino
    - Acurácia
    - Matriz de Confusão

In [ ]:
def validation(model, loader, criterion):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for data in loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs,labels)
            val_loss += loss.item()
    return val_loss/len(loader)

In [ ]:
def train(model, trainloader, testloader, optimizer, criterion, epochs):
    for epoch in range(epochs):
        model.train()
        running_loss = 0
        for data in tqdm(trainloader):
            images, labels = data
            model.zero_grad()
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        val_loss = validation(model, testloader, criterion)
        print(f'Epoch: {epoch+1} | Loss: {running_loss/len(trainloader)} | Val Loss: {val_loss}')

In [ ]:
def accuracy(model, loader):
    model.eval()
    corrected = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _,predicted = torch.max(outputs, 1)
            total += labels.size(0)
            corrected += (predicted == labels).sum().item()
    return corrected * 100 // total

In [ ]:
def confusion_matrix(model, loader):
    model.eval()
    confusion_matrix = np.zeros((2,2))
    with torch.no_grad():
        for data in loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _,predicted = torch.max(outputs, 1)
            for i in range(labels.size(0)):
                confusion_matrix[labels[i].item()][predicted[i].item()] += 1
    ax = sns.heatmap(confusion_matrix, annot=True, cmap='Blues', fmt='g', xticklabels=['Coral Falsa', 'Coral Verdadeira'], yticklabels=['Coral Falsa', 'Coral Verdadeira'])
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Label')
    return ax

---

# Treinando o Modelo

    - Utilizando a resnet
    - Congelar pesos das camadas
    - Treinar com o dados

In [ ]:
resnet = torchvision.models.resnet50(weights = torchvision.models.ResNet50_Weights.DEFAULT)

In [ ]:
resnet.fc = nn.Linear(2048,2)

In [ ]:
for name, params in resnet.named_parameters():
    if name not in ('fc.weight', 'fc.bias'):
        params.requires_grad = False

In [ ]:
resnet.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
epochs = 5

In [ ]:
train(resnet, train_loader, test_loader, optimizer, criterion, epochs)

In [ ]:
print(f'A rede atinge: {accuracy(resnet, test_loader)}% de acurácia')
conf_mat = confusion_matrix(resnet, test_loader)

---

# Comparando os Modelos

    - Primeiro nós precisamos fazer os outros modelos né kk


In [ ]:
def predict(model, image):
    prediction = model(torch.unsqueeze(image, 0).to(device))
    result = torch.argmax(prediction)
    return 'Coral Verdadeira' if result == 0 else 'Coral Falsa'

In [ ]:
image_index = 20
image = test[image_index][0]
label = 'Coral Falsa' if test[image_index][1] else 'Coral Verdadeira'
imshow(image)
print(label)

In [ ]:
print(f'Real Label: {label}')
print(f'Resnet Prediction: {predict(resnet, image)}')

In [ ]:
from PIL import Image
import torchvision.transforms.functional as TF

x = Image.open('teste.jpg').convert('RGB')
x = transformations(x)

In [ ]:
print(f'Resnet Prediction: {predict(resnet, x)}')